# Enunciado

El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en
Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: 
(fecha, id_caso,descripcion, estado_caso, categoria, latitud, longitud).
Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución. Las
fechas se encuentran en el formato YYYY-MM-DD.
Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para
pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato: 

(id_caso,respuesta), 

El campo respuesta depende de si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.
El sector encargado de las estadísticas oficiales del GCPD quiere con esta información analizar las siguientes
situaciones:
1) Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no
participó Batman).
2) Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la
batiseñal, participó en la resolución).
3) Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos.

In [1]:
import pandas as pd

In [2]:
casos = pd.read_csv('casos.csv', index_col=['id_caso'])
batiseñal=pd.read_csv('batisenial.csv',index_col=['id_caso'])

In [3]:
casos['estado_caso'].replace(1,'abierto', inplace=True)
casos['estado_caso'].replace(2,'resuelto', inplace=True)
casos['estado_caso'].replace(3,'sin_resolucion', inplace=True)
casos.head()

,fecha,descripcion,estado_caso,categoria,latitud,longitud
id_caso,,,,,,
1,2017-06-21,Desc1,abierto,Asesinato,40.75793,-73.98551
2,2017-06-26,Desc2,abierto,Robo,41.75213,-72.98285
3,2017-06-19,Desc3,resuelto,Fraude,39.37793,-73.18525
4,2017-06-16,Desc4,sin_resolucion,Asesinato,40.75724,-72.58912
5,2017-06-23,Desc5,resuelto,Fraude,40.71193,-73.59812


In [4]:
batiseñal.head()

,respuesta
id_caso,
1,1
2,1
3,1
6,0
7,1


# Punto 1

Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no
participó Batman)

La tasa de resolución se calcula como:

Nº Casos resueltos / (Nº Casos resueltos + Nº Casos abiertos + Nº Casos cerrados sin resolución)

Si además se quiere a la tasa de resolución expresada en % entonces simplemente se puede multiplicar por 100!

Notar que solo es necesario considerar aquellos casos en los que NO participó Batman, es decir, aquellos casos donde no se activó la batiseñal o bien donde se activó la batiseñal pero se obtuvo una respuesta negativa de Batman

In [5]:
casos_con_y_sin_batman = casos.merge(batiseñal, how='left', left_index=True, right_index=True)[['categoria', 'estado_caso','respuesta']]
casos_con_y_sin_batman.fillna(0, inplace=True)
casos_con_y_sin_batman.head()

,categoria,estado_caso,respuesta
id_caso,,,
1,Asesinato,abierto,1.0
2,Robo,abierto,1.0
3,Fraude,resuelto,1.0
4,Asesinato,sin_resolucion,0.0
5,Fraude,resuelto,0.0


In [6]:
#Me quiero quedar con aquellos casos donde la respuesta no es igual a 1
casos_sin_batman = casos_con_y_sin_batman[casos_con_y_sin_batman['respuesta'] != 1]
casos_sin_batman.head(20)

,categoria,estado_caso,respuesta
id_caso,,,
4,Asesinato,sin_resolucion,0.0
5,Fraude,resuelto,0.0
6,Asesinato,resuelto,0.0
9,Fraude,abierto,0.0
10,Asesinato,sin_resolucion,0.0
11,Robo,sin_resolucion,0.0
12,Asesinato,abierto,0.0
13,Asesinato,resuelto,0.0
15,Asesinato,abierto,0.0


In [7]:
#Ahora obtengo la cantidad de casos que hay por categoria y estado caso
#Notar que el groupby genera un indice multiple por filas compuesto por (categoria,estado_caso)
#También genera otro indice por columna a través del campo respuesta
cantidad_por_categoria_y_caso_sin_batman=casos_sin_batman.groupby(['categoria', 'estado_caso']).count()
cantidad_por_categoria_y_caso_sin_batman.head()

respuesta
categoria estado_caso              
Asesinato abierto                 2
          resuelto                4
          sin_resolucion          2
Fraude    abierto                 2
          resuelto                1

In [8]:
#Si quisiera transformar estado_caso en columnas, puedo aplicar unstack
#Por defecto, unstack tomará el último nivel del índice (categoria, estado_caso). Es decir, tomará estado_caso.
#Al aplicar unstack, se creará un índice múltiple por columna compuesto por (respuesta, estado_caso)
#Ver doc: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html
dataframe_final_sin_batman = cantidad_por_categoria_y_caso_sin_batman.unstack(fill_value=0)
dataframe_final_sin_batman.head()

respuesta                        
estado_caso   abierto resuelto sin_resolucion
categoria                                    
Asesinato           2        4              2
Fraude              2        1              0
Robo                0        0              1

In [9]:
#Se puede notar en el head anterior que el nivel del índice llamado "respuesta" no lo necesitamos.
#Se lo puede quitar de la siguiente forma
dataframe_final_sin_batman.columns = dataframe_final_sin_batman.columns.droplevel(0)
dataframe_final_sin_batman.head()

estado_caso,abierto,resuelto,sin_resolucion
categoria,,,
Asesinato,2,4,2
Fraude,2,1,0
Robo,0,0,1


In [10]:
#Para calcular la tasa, simplemente hacemos 100*resuelto / (resuelto+abierto+sin_resolucion)
dataframe_final_sin_batman['tasa_resolucion'] = 100*dataframe_final_sin_batman['resuelto']/(dataframe_final_sin_batman['resuelto']+dataframe_final_sin_batman['abierto']+dataframe_final_sin_batman['sin_resolucion'])
dataframe_final_sin_batman.head()

estado_caso,abierto,resuelto,sin_resolucion,tasa_resolucion
categoria,,,,
Asesinato,2,4,2,50.000000
Fraude,2,1,0,33.333333
Robo,0,0,1,0.000000


# Punto 2

Para calcular la tasa de resolucion en donde sí participó batman, se puede hacer simplemente lo mismo que antes

In [11]:
#Me quiero quedar con aquellos casos donde la respuesta no es igual a 1
casos_con_batman = casos_con_y_sin_batman[casos_con_y_sin_batman['respuesta'] == 1]
dataframe_final_con_batman = casos_con_batman.groupby(['categoria', 'estado_caso']).count().unstack(fill_value=0)
dataframe_final_con_batman.columns = dataframe_final_con_batman.columns.droplevel(0)
dataframe_final_con_batman['tasa_resolucion'] = 100*dataframe_final_con_batman['resuelto']/(dataframe_final_con_batman['resuelto']+dataframe_final_con_batman['abierto']+dataframe_final_con_batman['sin_resolucion'])
dataframe_final_con_batman.head()

estado_caso,abierto,resuelto,sin_resolucion,tasa_resolucion
categoria,,,,
Asesinato,1,0,0,0.0
Fraude,1,1,0,50.0
Robo,2,1,1,25.0


# Punto 3

Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos

Este punto se podría entender como el mes para el cual la cantidad de casos en la que estuvo es máxima

Para ello, podemos filtrar el dataframe por año 2017, agrupar por mes calculando la cantidad y ordenándolo.

In [12]:
casos['fecha'] = pd.to_datetime(casos['fecha'])
meses_2017 = casos[casos['fecha'].dt.year==2017][['fecha']]
meses_2017['mes'] = meses_2017['fecha'].dt.month
meses_2017.head()

,fecha,mes
id_caso,,
1,2017-06-21,6
2,2017-06-26,6
3,2017-06-19,6
4,2017-06-16,6
5,2017-06-23,6


In [13]:
#Asumiendo que el campo feha no es nulo, puedo hacer un count sobre dicho campo y con eso puedo calcular
#la cantidad de casos en los que participó batman por mes
meses_2017.groupby('mes').count().sort_values(by='fecha', ascending=False).head(1)

,fecha
mes,
7,10


In [14]:
#Otra forma más sencilla de resolver el punto anterior podría ser la siguiente:
meses_2017['mes'].value_counts().nlargest(1)

7    10
Name: mes, dtype: int64